In [18]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs 
import pandas as pd
import re
import time
import sys

In [5]:
# 화면을 아래쪽으로 스크롤
def pang_Down():
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

In [24]:
keyword = input("검색어 입력:")
cntdown = int(input("스크롤 다운 건수:"))

driver=Chrome("./data/chromedriver.exe")

# 인스타그램 접속
driver.get("https://www.instagram.com/")
time.sleep(2)

# 인스타그램 로그인
userID=""
userPW=""

# 아이디 입력
input_id=driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(1) > div > label > input")
input_id.send_keys(userID)

# 패스워드 입력
input_pw=driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(2) > div > label > input")
input_pw.send_keys(userPW)

# 로그인 정보 전송 / 로그인 버튼을 클릭해도 됨
input_pw.submit()  
time.sleep(3)

state = input("로그인 되었습니까?[y/n]:")
if state == "n" or state == "ㅜ" or state == "N":
    print("작업을 종료합니다.")
    driver.close()
    sys.exit()

검색어 입력:제주도맛집
스크롤 다운 건수:5


C:\Users\user01\AppData\Local\Temp/ipykernel_16540/719783206.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=Chrome("./data/chromedriver.exe")


로그인 되었습니까?[y/n]:y


In [25]:
# 인스타 검색 url 확인 및 사용자 키워드 적용

url="https://www.instagram.com/explore/tags/" + keyword
driver.get(url)
time.sleep(5)

body = driver.find_element(By.TAG_NAME, "body")

In [27]:
# 하이퍼 링크 주소 추출
link_url = []

for down in range(cntdown):
    pang_Down()
    #content = body.find_element_by_class_name("v1Nh3")
    contents = body.find_elements(By.CLASS_NAME, "v1Nh3")
    
    # 게시글 리스트 URL 추출
    for content in contents:
        url = content.find_element(By.TAG_NAME, "a").get_attribute("href")
        link_url.append(url)
        
link_url=list(set(link_url))  # 중복 데이터 제거

['https://www.instagram.com/p/CbqsyZ6BIm-/',
 'https://www.instagram.com/p/CbqxeCshE2y/',
 'https://www.instagram.com/p/CboUX-2rINL/',
 'https://www.instagram.com/p/CbqsyYOBjKe/',
 'https://www.instagram.com/p/CbqsybIhLct/',
 'https://www.instagram.com/p/CbqsyaJBn-D/',
 'https://www.instagram.com/p/CbobuG4BEPZ/',
 'https://www.instagram.com/p/CbogW3kv3di/',
 'https://www.instagram.com/p/CbojUN_hnHm/',
 'https://www.instagram.com/p/CbrLs6XpcXZ/',
 'https://www.instagram.com/p/CbrLgXTviWb/',
 'https://www.instagram.com/p/CbrLfrRhPW_/',
 'https://www.instagram.com/p/CbrLmxSJq1c/',
 'https://www.instagram.com/p/CbrLOZ6pyMR/',
 'https://www.instagram.com/p/CbrLV8DLZ2V/',
 'https://www.instagram.com/p/CbrLZMwrh04/',
 'https://www.instagram.com/p/CbrLbgvvMdS/',
 'https://www.instagram.com/p/CbrLIlbrZgg/',
 'https://www.instagram.com/p/CbrLBm2PZ3B/',
 'https://www.instagram.com/p/CbrKvBxPB6k/',
 'https://www.instagram.com/p/CbrKvXBvb-m/',
 'https://www.instagram.com/p/CbrKuY0roQo/',
 'https://

링크를 이용해 데이터 가져오기

In [38]:
conttxt = []
#for link in link_url:
    
for i in range(5):    
    driver.get(link_url[i])
    time.sleep(2)

    html =  driver.page_source
    soup = bs(html, "html.parser")
    conttxt.append(soup.find("div", class_="C4VMK").find_all("span")[1].text)

In [42]:
# 정규식 표현을 이용한 불필요한 데이터 제거하기
re.sub('[^0-9a-zA-Zㄱ-ㅣ가-힣#?! ]' ,"", conttxt[0])

'제주 소금바치순이네불향가득한 매콤한 돌문어볶음!!소면에 양념밥돌문어 같이 먹으면 대박임도민들도 추천하는 맛집이야!같이 갈 사람 태그주소  제주시 구좌읍 해맞이해안로 2196#제주맛집 #제주도맛집 #제주공항근처맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플'

아이디, 본문, 해시태그, 좋아요, 댓글 가져와 DataFrame으로 출력

In [44]:
# 아이디 가져오기
soup.find("div", class_="C4VMK").find("a").get_text()

'travel_seasia'

In [51]:
# 본문 가져오기
soup.find('div', class_="MOdxS").find('span').text.split("#")[0]

'제주도 동쪽 갈치 1등.jpg하루에 갈치만 150마리 판다고 함..ㄷㄷㄷㄷㄷㄷㄷㄷ히밥이 먹고 극찬한 보리김치도 여기 꺼였음!!+ 고등어조림이랑 같이 나오는 세트 강츄⠀출처: @muk__dam⠀'

In [53]:
# 해시태그 가져오기
tags = soup.find('div', class_="MOdxS").find_all('a')
for tag in tags:
    if "#" in tag.text:
        print(tag.text)

#제주맛집
#성산갈치맛집
#동행스타그램


In [71]:
# 댓글 가져오기
li_soup=soup.find("div", class_="EtaWk").find_all("li", class_="gElp9 rUo9f")[2]
li_soup.find_all("a")[1].text
li_soup.find("span").text

'seoungyunnni'

### url을 이용해 인스타그램 정보 크롤
- 아이디, 내용, 해시태그
- 잘 되시면 repuests 모듈로도 적용해 보세요